# Description 
This script scrapes the stoiximan webpage. It provides a dataframe of all the upcoming EPL matches that stoiximan provides together with the odds of the Betting company. It then joins them with the 'GatheredData/eplGMTStdTimeFiles/epl-2023-GMTStandardTime.csv' file and makes a 'Newfixtures.csv' file. In the end it runs all the needed scripts so that the predictor runs correctly for each round.

In [5]:
import requests 
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import datetime

url = 'https://en.stoiximan.gr/sport/soccer/england/premier-league/1/'
page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

In [6]:
# A function to change team names as depicted in Stoiximan in the way they are depicted in the files
def replace_names(string):
    old_names = ['Fulham FC', 'Everton FC', 'Leeds United', 'Southampton FC', 'West Ham United', 'Arsenal FC', 'AFC Bournemouth', 'Crystal Palace FC', 'Liverpool FC', 'Brighton & Hove Albion', 'Chelsea FC', 'Tottenham Hotspur', 'Brentford FC', 'Burnley FC', 'Luton Town']
    new_names = ['Fulham', 'Everton', 'Leeds', 'Southampton', 'West Ham', 'Arsenal', 'Bournemouth', 'Crystal Palace', 'Liverpool', 'Brighton', 'Chelsea', 'Tottenham', 'Brentford', 'Burnley', 'Luton']
    
    for i in range(len(old_names)):
        string = string.replace(old_names[i], new_names[i])
    return string

In [7]:
scripts = soup.find_all("script")

In [8]:
data = []
for script in scripts:
    data.extend(script)

Odds = []
Dates = []
HomeTeams = []
AwayTeams = []

data = data[3]
data = re.sub(r"window\[\"initial_state\"\]=", "", data)
print(data)
d = json.loads(data)


for match in d['data']['blocks'][0]['events']:
    teams = match['shortName'].split('-')
    HomeTeams.append(replace_names(teams[0]))
    AwayTeams.append(replace_names(teams[1]))
    Dates.append(datetime.datetime.fromtimestamp(match['startTime']/1000.0).strftime('%d/%m/%Y'))
    for x in match['markets']:
        for y in x['selections']:
            Odds.append(y['price'])
            


rows = []
while Odds:
    rows.append(Odds[:7])
    Odds = Odds[7:]

print(rows)
print(Dates)

{"data":{"leagueStandings":[],"sport":{"id":"FOOT","name":"Soccer","region":"England","url":"/sport/soccer/"},"markets":[{"id":"0","name":"Main","selected":true,"type":"Main"},{"id":"1","name":"Double chance"},{"id":"2","name":"Half Time"},{"id":"3","name":"Asian Lines"},{"id":"4","name":"Corner kicks"},{"id":"5","name":"Cards"},{"id":"6","name":"Players","type":"Players"},{"id":"7","name":"Players' specials"},{"id":"8","name":"Long term bets"}],"blocks":[{"name":"England - Premier League","shortName":"Premier League","id":"1r","url":"/sport/soccer/england/premier-league/1/","headers":[{"marketTypes":["MR12","MRES"],"columns":[{"columnSpan":1,"name":"1"},{"columnSpan":1,"name":"X"},{"columnSpan":1,"name":"2"}]},{"marketTypes":["HCTG"],"columns":[{"columnSpan":1,"name":"OVER/UNDER"}]},{"marketTypes":["BTSC"],"columns":[{"columnSpan":1,"name":"GG/NG"}]}],"layout":2,"events":[{"stats":[{"url":"https://s5.sir.sportradar.com/stoiximan/en/match/41762837","providerId":1}],"sportId":"FOOT","sh

In [9]:
df_teams = pd.DataFrame({'Home Team': HomeTeams, 'Away Team': AwayTeams})
df_values = pd.DataFrame(rows, columns=['1', 'x', '2', 'Over', 'Under', 'Yes', 'No'])
df_all = pd.concat([df_teams, df_values], axis=1)
df_all['Home Team'] = df_all['Home Team'].str.strip()
df_all['Away Team'] = df_all['Away Team'].str.strip()
display(df_all)

,Home Team,Away Team,1,x,2,Over,Under,Yes,No
0,Burnley,Manchester City,9.00,5.40,1.33,1.62,2.32,2.00,1.75
1,Arsenal,Nottingham Forest,1.20,6.80,14.50,2.07,1.75,2.05,1.72
2,Brighton,Luton,1.33,5.30,9.50,1.62,2.30,1.98,1.78
3,Everton,Fulham,2.20,3.40,3.30,1.95,1.85,1.72,2.05
4,Sheffield United,Crystal Palace,3.10,3.25,2.40,2.18,1.70,1.87,1.87
5,Bournemouth,West Ham,2.72,3.40,2.60,1.88,1.93,1.65,2.15
6,Newcastle United,Aston Villa,1.75,3.95,4.40,1.75,2.07,1.72,2.05
7,Brentford,Tottenham,2.72,3.65,2.45,1.72,2.15,1.57,2.30
8,Chelsea,Liverpool,2.85,3.60,2.37,1.70,2.18,1.55,2.35
9,Manchester United,Wolverhampton Wanderers,1.35,5.20,8.50,1.62,2.30,1.93,1.82


In [10]:
df = pd.read_csv('GatheredData/eplGMTStdTimeFiles/epl-2023-GMTStandardTime.csv')

display(df)

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
0,1,1,11/08/2023 20:00,Turf Moor,Burnley,Man City,NaN
1,2,1,12/08/2023 12:30,Emirates Stadium,Arsenal,Nottingham Forest,NaN
2,3,1,12/08/2023 15:00,Vitality Stadium,Bournemouth,West Ham,NaN
3,4,1,12/08/2023 15:00,Amex Stadium,Brighton,Luton,NaN
4,5,1,12/08/2023 15:00,Goodison Park,Everton,Fulham,NaN
...,...,...,...,...,...,...,...
375,376,38,19/05/2024 16:00,Selhurst Park,Crystal Palace,Aston Villa,NaN
376,377,38,19/05/2024 16:00,Anfield,Liverpool,Wolves,NaN
377,378,38,19/05/2024 16:00,Kenilworth Road,Luton,Fulham,NaN
378,379,38,19/05/2024 16:00,Etihad Stadium,Man City,West Ham,NaN


In [11]:
df_merged = pd.merge(df, df_all, on=['Home Team', 'Away Team'], how='right')
df_merged['Date']=Dates
display(df_merged)
df_merged.to_csv('Newfixtures.csv')

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result,1,x,2,Over,Under,Yes,No
0,NaN,NaN,11/08/2023,NaN,Burnley,Manchester City,NaN,9.00,5.40,1.33,1.62,2.32,2.00,1.75
1,2.0,1.0,12/08/2023,Emirates Stadium,Arsenal,Nottingham Forest,NaN,1.20,6.80,14.50,2.07,1.75,2.05,1.72
2,4.0,1.0,12/08/2023,Amex Stadium,Brighton,Luton,NaN,1.33,5.30,9.50,1.62,2.30,1.98,1.78
3,5.0,1.0,12/08/2023,Goodison Park,Everton,Fulham,NaN,2.20,3.40,3.30,1.95,1.85,1.72,2.05
4,NaN,NaN,12/08/2023,NaN,Sheffield United,Crystal Palace,NaN,3.10,3.25,2.40,2.18,1.70,1.87,1.87
5,3.0,1.0,12/08/2023,Vitality Stadium,Bournemouth,West Ham,NaN,2.72,3.40,2.60,1.88,1.93,1.65,2.15
6,NaN,NaN,12/08/2023,NaN,Newcastle United,Aston Villa,NaN,1.75,3.95,4.40,1.75,2.07,1.72,2.05
7,NaN,NaN,13/08/2023,NaN,Brentford,Tottenham,NaN,2.72,3.65,2.45,1.72,2.15,1.57,2.30
8,9.0,1.0,13/08/2023,Stamford Bridge,Chelsea,Liverpool,NaN,2.85,3.60,2.37,1.70,2.18,1.55,2.35
9,NaN,NaN,14/08/2023,NaN,Manchester United,Wolverhampton Wanderers,NaN,1.35,5.20,8.50,1.62,2.30,1.93,1.82


In [12]:
import os

# run the script named "my_script.py" with argument "arg1"
os.system("python NewFixturesScraping2.py")
os.system("python NewfixturesScraping2Month.py")
os.system("python NewfixturesScraping2Month3.py")
os.system("python Gather2022Season.py")
os.system("python Gather2022SeasonForm.py")
os.system("python Gather2022SeasonForm_3Month.py")

11/08/2023
0    Burnley
Name: Home Team, dtype: object
0    Manchester City
Name: Away Team, dtype: object


/home/nickpad/Documents/EplProject/NewFixturesScraping2.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(


12/08/2023
1             Arsenal
2            Brighton
3             Everton
4    Sheffield United
5         Bournemouth
6    Newcastle United
Name: Home Team, dtype: object
1    Nottingham Forest
2                Luton
3               Fulham
4       Crystal Palace
5             West Ham
6          Aston Villa
Name: Away Team, dtype: object


/home/nickpad/Documents/EplProject/NewFixturesScraping2.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EplProject/NewFixturesScraping2.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(


13/08/2023
7    Brentford
8      Chelsea
Name: Home Team, dtype: object
7    Tottenham
8    Liverpool
Name: Away Team, dtype: object


...

20/08/2023
16    Aston Villa
17       West Ham
Name: Home Team, dtype: object
16    Everton
17    Chelsea
Name: Away Team, dtype: object


/home/nickpad/Documents/EplProject/NewfixturesScraping2Month3.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EplProject/NewfixturesScraping2Month3.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(


21/08/2023
18    Crystal Palace
Name: Home Team, dtype: object
18    Arsenal
Name: Away Team, dtype: object
                      HTeam  HM  HG  ...        HDC       HODC    HPTS/M
0                   Arsenal   3   5  ...  10.333333   3.333333  1.000000
1                  Brighton   5   9  ...   7.000000   8.600000  1.400000
2                   Everton   3   2  ...   4.666667   6.333333  1.333333
3               Bournemouth   3   0  ...   5.666667   8.333333  0.000000
4          Newcastle United   4   7  ...  10.500000   6.000000  1.500000
5                 Brentford   3   6  ...   8.000000   5.333333  3.000000
6                   Chelsea   4   4  ...  11.250000   9.250000  0.500000
7         Manchester United   4   9  ...  14.250000   7.750000  3.000000
8         Nottingham Forest   2   2  ...   3.000000  11.000000  2.000000
9   Wolverhampton Wanderers   2   1  ...   4.500000  10.500000  0.500000
10                Liverpool   2   5  ...  15.000000   3.000000  1.000000
11              

/home/nickpad/Documents/EplProject/NewfixturesScraping2Month3.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EplProject/NewfixturesScraping2Month3.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(
python: can't open file '/home/nickpad/Documents/EplProject/Gather2022Season.py': [Errno 2] No such file or directory
python: can't open file '/home/nickpad/Documents/EplProject/Gather2022SeasonForm.py': [Errno

512